Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
g = tf.Graph()
batch_size = 128
num_hidden = 1024

decay = 0.01

with g.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    W_1 = tf.Variable(tf.truncated_normal([image_size*image_size, num_hidden]))
    B_1 = tf.Variable(tf.zeros([num_hidden]))
    
    W_2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    B_2 = tf.Variable(tf.zeros([num_labels]))
    
    O_1 = tf.nn.relu(tf.matmul(tf_train_dataset,W_1) + B_1)
    O_2 = tf.matmul(O_1,W_2) + B_2
    
    loss =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(O_2, tf_train_labels)) \
        + decay * tf.nn.l2_loss(W_1) \
        + decay * tf.nn.l2_loss(W_2)
        
        
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(O_2)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,W_1)+B_1),W_2) + B_2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,W_1)+B_1),W_2) + B_2)

In [6]:
num_steps = 3001
with tf.Session(graph=g) as session:
    tf.initialize_all_variables().run()
    print("Initialization Complete")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        _,l,predictions = session.run([optimizer,loss,train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialization Complete
Minibatch loss at step 0: 3459.722656
Minibatch accuracy: 8.6%
Validation accuracy: 25.2%
Minibatch loss at step 500: 21.192955
Minibatch accuracy: 89.8%
Validation accuracy: 83.8%
Minibatch loss at step 1000: 1.063792
Minibatch accuracy: 78.9%
Validation accuracy: 82.8%
Minibatch loss at step 1500: 0.735935
Minibatch accuracy: 83.6%
Validation accuracy: 83.7%
Minibatch loss at step 2000: 0.808036
Minibatch accuracy: 82.8%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 0.750041
Minibatch accuracy: 78.1%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 0.675492
Minibatch accuracy: 86.7%
Validation accuracy: 83.2%
Test accuracy: 89.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
num_steps = 1001
with tf.Session(graph=g) as session:
    tf.initialize_all_variables().run()
    print("Initialization Complete")
    for step in range(num_steps):
        offset = 0 #overfitting to data
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        _,l,predictions = session.run([optimizer,loss,train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialization Complete
Minibatch loss at step 0: 3484.982422
Minibatch accuracy: 10.9%
Validation accuracy: 31.1%
Minibatch loss at step 500: 21.027145
Minibatch accuracy: 100.0%
Validation accuracy: 66.2%
Minibatch loss at step 1000: 0.343473
Minibatch accuracy: 100.0%
Validation accuracy: 67.9%
Test accuracy: 74.3%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
g = tf.Graph()
batch_size = 128
num_hidden = 1024

decay = 0.01

with g.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    W_1 = tf.Variable(tf.truncated_normal([image_size*image_size, num_hidden]))
    B_1 = tf.Variable(tf.zeros([num_hidden]))
    
    W_2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    B_2 = tf.Variable(tf.zeros([num_labels]))
    
    O_1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset,W_1) + B_1), 0.5) #0.5 = probability of dropout
    O_2 = tf.matmul(O_1,W_2) + B_2
    
    loss =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(O_2, tf_train_labels)) \
        + decay * tf.nn.l2_loss(W_1) \
        + decay * tf.nn.l2_loss(W_2)
        
        
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(O_2)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,W_1)+B_1),W_2) + B_2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,W_1)+B_1),W_2) + B_2)

In [11]:
import random
num_steps = 3001
with tf.Session(graph=g) as session:
    tf.initialize_all_variables().run()
    print("Initialization Complete")
    for step in range(num_steps):
        offset = random.choice([0,1,2]) #small pool, extreme overfitting
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        _,l,predictions = session.run([optimizer,loss,train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialization Complete
Minibatch loss at step 0: 3708.563477
Minibatch accuracy: 10.2%
Validation accuracy: 20.9%
Minibatch loss at step 500: 21.045885
Minibatch accuracy: 100.0%
Validation accuracy: 65.8%
Minibatch loss at step 1000: 0.358846
Minibatch accuracy: 100.0%
Validation accuracy: 68.7%
Minibatch loss at step 1500: 0.206513
Minibatch accuracy: 100.0%
Validation accuracy: 68.8%
Minibatch loss at step 2000: 0.198550
Minibatch accuracy: 100.0%
Validation accuracy: 69.1%
Minibatch loss at step 2500: 0.199843
Minibatch accuracy: 100.0%
Validation accuracy: 69.3%
Minibatch loss at step 3000: 0.193328
Minibatch accuracy: 100.0%
Validation accuracy: 69.6%
Test accuracy: 76.3%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [20]:
g = tf.Graph()
batch_size = 256
num_hidden = 1024
decay = 0.001
dropout_p = 0.5
num_steps = 10001

def forward(X,W_1,B_1,W_2,B_2):
    O_1 = tf.nn.relu(tf.matmul(X,W_1) + B_1)
    O_2 = tf.matmul(O_1,W_2) + B_2
    return O_2

with g.as_default():
    tf_batch_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size*image_size))
    tf_batch_labels = tf.placeholder(tf.float32,shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    W_1 = tf.Variable(tf.truncated_normal([image_size*image_size, num_hidden]))
    B_1 = tf.Variable(tf.zeros([num_hidden]))
    
    W_2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    B_2 = tf.Variable(tf.zeros([num_labels]))
    
    O_1 = tf.nn.relu(tf.matmul(tf_batch_dataset, W_1) + B_1)
    O_d = tf.nn.dropout(O_1, dropout_p)
    O = tf.matmul(O_d, W_2) + B_2
    global_step = tf.Variable(0)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(O, tf_batch_labels)) + \
            decay * tf.nn.l2_loss(W_1) + \
            decay * tf.nn.l2_loss(W_2)
    learning_rate = tf.train.exponential_decay(0.5,global_step,decay_steps = num_steps, decay_rate = 0.95)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step = global_step)
    #optimizer = tf.train.AdagradOptimizer(learning_rate=0.1, initial_accumulator_value=0.1).minimize(loss)
    optimizer = tf.train.AdadeltaOptimizer(learning_rate=0.5, rho=0.95, epsilon=1e-9).minimize(loss)
    train_prediction = tf.nn.softmax(O)
    valid_prediction = tf.nn.softmax(forward(tf_valid_dataset,W_1,B_1,W_2,B_2))
    test_prediction = tf.nn.softmax(forward(tf_test_dataset,W_1,B_1,W_2,B_2))
    

In [21]:

with tf.Session(graph=g) as session:
    tf.initialize_all_variables().run()
    print ("Initialized")
    for step in range(num_steps):
        #print(step)
        indices = np.random.randint(train_dataset.shape[0], size=batch_size)
        batch_dataset = train_dataset[indices]
        batch_labels = train_labels[indices]
        feed_dict = {tf_batch_dataset : batch_dataset, tf_batch_labels : batch_labels}
        _,l,predictions = session.run([optimizer,loss,train_prediction], feed_dict = feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
            

Initialized
Minibatch loss at step 0: 887.765808
Minibatch accuracy: 8.6%
Validation accuracy: 5.9%
Minibatch loss at step 500: 199.483551
Minibatch accuracy: 80.1%
Validation accuracy: 81.8%
Minibatch loss at step 1000: 119.544037
Minibatch accuracy: 78.5%
Validation accuracy: 80.1%
Minibatch loss at step 1500: 70.808762
Minibatch accuracy: 82.0%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 43.106926
Minibatch accuracy: 80.1%
Validation accuracy: 84.3%
Minibatch loss at step 2500: 26.484665
Minibatch accuracy: 81.2%
Validation accuracy: 85.2%
Minibatch loss at step 3000: 16.169365
Minibatch accuracy: 83.6%
Validation accuracy: 85.9%
Minibatch loss at step 3500: 10.067521
Minibatch accuracy: 86.3%
Validation accuracy: 86.9%
Minibatch loss at step 4000: 6.289770
Minibatch accuracy: 87.1%
Validation accuracy: 87.1%
Minibatch loss at step 4500: 4.008841
Minibatch accuracy: 89.1%
Validation accuracy: 87.6%
Minibatch loss at step 5000: 2.682217
Minibatch accuracy: 85.9%
Validatio